# Interval statistics algorithm — methods reference

This notebook is a **standalone narrative description** of the interval statistics algorithm in `intv_stats.py`. It reproduces the exact logic using pandas and numpy only. The pipeline:

1. **Filter** the master dataframe by roi_id, rel_path, and event_type.
2. **Extract the time series** (e.g. t_start) from the filtered rows.
3. **Compute iei** (inter-event-interval) = diff of successive event times.
4. **Compute inst_freq** (instantaneous frequency) = 1/iei.
5. **Step 2.5**: Filter out iei=0 (detection errors); record n_original.
6. **Aggregate** count, min, max, mean, std, sem, cv over filtered iei and inst_freq; add n_original column.
7. **Parse rel_path** into grandparent, parent, tif_file for the final table.

All intermediate results are displayed as pandas DataFrames.

## Important assumptions

1. **Ordering**: Within a filtered group (roi_id, rel_path), events are assumed to be **already ordered chronologically**. The algorithm does not sort.
2. **Zero IEI (Step 2.5)**: Successive events may share the same timestamp (iei=0), assumed to be detection errors (two events cannot occur at the same time). We **filter them out** before aggregation. n_original = count before filtering; count = count after filtering.

## Setup

Imports, path to the example CSV, and algorithm parameters. Run from the **nicewidgets project root**.

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Any

DATA_DIR = Path("data")
if not (DATA_DIR / "kym_event_report.csv").exists():
    DATA_DIR = Path("../data")
CSV_PATH = DATA_DIR / "kym_event_report.csv"
assert CSV_PATH.exists(), f"Example CSV not found: {CSV_PATH}"

TIME_COL = "t_start"
ROI_ID = 1
REL_PATH = "14d Saline/20251020/20251020_A100_0013.tif"
EVENT_TYPE = "baseline_rise"

## Algorithm functions (from intv_stats.py)

The following cells define the same functions as in `intv_stats.py` — the notebook is self-contained.

### parse_rel_path

Split rel_path by '/' into grandparent, parent, tif_file. Example: `'14d Saline/20251020/20251020_A100_0013.tif'` → grandparent='14d Saline', parent='20251020', tif_file='20251020_A100_0013.tif'.

In [2]:
def parse_rel_path(rel_path: str) -> dict[str, str]:
    parts = rel_path.strip().split("/")
    if len(parts) == 0:
        return {"grandparent": "", "parent": "", "tif_file": ""}
    if len(parts) == 1:
        return {"grandparent": parts[0], "parent": "", "tif_file": parts[0]}
    if len(parts) == 2:
        return {"grandparent": parts[0], "parent": "", "tif_file": parts[1]}
    return {
        "grandparent": parts[0],
        "parent": parts[1],
        "tif_file": parts[-1],
    }

### filter_for_intv_stats

Filter the master dataframe by roi_id, rel_path, and event_type. All three are required.

In [3]:
def filter_for_intv_stats(
    df: pd.DataFrame,
    roi_id: Any,
    rel_path: str,
    event_type: str,
) -> pd.DataFrame:
    df_f = df.copy()
    df_f = df_f[df_f["roi_id"].astype(str) == str(roi_id)]
    df_f = df_f[df_f["rel_path"].astype(str) == str(rel_path)]
    df_f = df_f[df_f["event_type"].astype(str) == str(event_type)]
    return df_f.reset_index(drop=True)

### compute_iei_and_inst_freq

iei = diff(ts), inst_freq = 1/iei. First event gets nan. Zero IEI → inst_freq = inf (filtered in Step 2.5).

In [4]:
def compute_iei_and_inst_freq(ts: pd.Series) -> tuple[pd.Series, pd.Series]:
    ts = pd.to_numeric(ts, errors="coerce")
    iei = ts.diff()
    inst_freq = 1.0 / iei
    return iei, inst_freq

### filter_zero_iei (Step 2.5)

Remove iei=0 (and corresponding inst_freq) before aggregation. We assume iei=0 indicates detection errors. Returns filtered series and n_original = count of non-NaN iei before filtering.

In [5]:
def filter_zero_iei(iei: pd.Series, inst_freq: pd.Series) -> tuple[pd.Series, pd.Series, int]:
    iei_f = iei.copy()
    inst_freq_f = inst_freq.copy()
    n_original = iei.notna().sum()
    mask = iei == 0
    iei_f = iei_f.where(~mask, np.nan)
    inst_freq_f = inst_freq_f.where(~mask, np.nan)
    return iei_f, inst_freq_f, int(n_original)

### aggregate_intv_stats

Compute count, min, max, mean, std, sem, cv for iei and inst_freq. Add n_original column.

In [6]:
AGG_STATS = ["count", "min", "max", "mean", "std", "sem", "cv"]

def aggregate_intv_stats(iei: pd.Series, inst_freq: pd.Series, n_original: int, cv_epsilon: float = 1e-10) -> pd.DataFrame:
    def _agg(s: pd.Series) -> dict[str, float]:
        valid = s.dropna()
        if len(valid) == 0:
            return {k: np.nan for k in AGG_STATS}
        mean_ = valid.mean()
        std_ = valid.std(ddof=1)
        cv_ = np.nan if np.abs(mean_) < cv_epsilon else (std_ / mean_)
        return {
            "count": valid.count(),
            "min": valid.min(),
            "max": valid.max(),
            "mean": mean_,
            "std": std_,
            "sem": valid.sem(ddof=1),
            "cv": cv_,
        }
    table = pd.DataFrame(
        [_agg(iei), _agg(inst_freq)],
        index=["iei", "inst_freq"],
        columns=AGG_STATS,
    )
    table["n_original"] = n_original
    return table

---
## Step 0: Master dataframe (raw)

Load the CSV and show shape and relevant columns.

In [7]:
df_master = pd.read_csv(CSV_PATH)
print("Shape:", df_master.shape)
cols = ["roi_id", "rel_path", "event_type", TIME_COL]
df_master[cols].head(15)

Shape: (116, 21)


,roi_id,rel_path,event_type,t_start
0,1,14d Saline/20251020/20251020_A100_0013.tif,baseline_rise,0.010
1,1,14d Saline/20251020/20251020_A100_0013.tif,baseline_rise,0.010
2,1,14d Saline/20251020/20251020_A100_0013.tif,baseline_rise,0.010
3,1,14d Saline/20251020/20251020_A100_0013.tif,baseline_rise,0.010
4,1,14d Saline/20251020/20251020_A100_0013.tif,baseline_rise,0.010
5,1,14d Saline/20251020/20251020_A100_0013.tif,nan_gap,0.378
6,1,14d Saline/20251020/20251020_A100_0013.tif,nan_gap,3.203
7,1,14d Saline/20251020/20251020_A100_0013.tif,baseline_drop,5.372
8,1,14d Saline/20251020/20251020_A100_0013.tif,baseline_drop,8.007
9,1,14d Saline/20251020/20251020_A100_0013.tif,nan_gap,8.098


## Step 1: After filter (roi_id, rel_path, event_type)

Filter the master dataframe to one roi_id, one rel_path, one event_type. This yields the time series of events for a single kym tif file.

In [8]:
df_f = filter_for_intv_stats(df_master, ROI_ID, REL_PATH, EVENT_TYPE)
print("Shape after filter:", df_f.shape)
print("Filter: roi_id=", ROI_ID, ", rel_path=", REL_PATH, ", event_type=", EVENT_TYPE)
df_f[["roi_id", "rel_path", "event_type", TIME_COL]]

Shape after filter: (7, 21)
Filter: roi_id= 1 , rel_path= 14d Saline/20251020/20251020_A100_0013.tif , event_type= baseline_rise


,roi_id,rel_path,event_type,t_start
0,1,14d Saline/20251020/20251020_A100_0013.tif,baseline_rise,0.010
1,1,14d Saline/20251020/20251020_A100_0013.tif,baseline_rise,0.010
2,1,14d Saline/20251020/20251020_A100_0013.tif,baseline_rise,0.010
3,1,14d Saline/20251020/20251020_A100_0013.tif,baseline_rise,0.010
4,1,14d Saline/20251020/20251020_A100_0013.tif,baseline_rise,0.010
5,1,14d Saline/20251020/20251020_A100_0013.tif,baseline_rise,21.551
6,1,14d Saline/20251020/20251020_A100_0013.tif,baseline_rise,21.551


## Step 2: Time series and iei, inst_freq

Extract the time column and compute iei = diff(ts), inst_freq = 1/iei. First event gets nan. Convention: iei[i] and inst_freq[i] are w.r.t. the previous event.

In [9]:
ts = df_f[TIME_COL]
iei, inst_freq = compute_iei_and_inst_freq(ts)
step2 = pd.DataFrame({
    TIME_COL: ts.values,
    "iei": iei.values,
    "inst_freq": inst_freq.values,
})
step2

,t_start,iei,inst_freq
0,0.010,NaN,NaN
1,0.010,0.000,inf
2,0.010,0.000,inf
3,0.010,0.000,inf
4,0.010,0.000,inf
5,21.551,21.541,0.046423
6,21.551,0.000,inf


## Step 2.5: Filter out iei=0 (detection errors)

Remove iei=0 and corresponding inst_freq before aggregation. n_original = number of intervals before filtering.

In [10]:
iei, inst_freq, n_original = filter_zero_iei(iei, inst_freq)
print("n_original =", n_original)
step2_5 = pd.DataFrame({
    TIME_COL: ts.values,
    "iei (filtered)": iei.values,
    "inst_freq (filtered)": inst_freq.values,
})
step2_5

n_original = 6


,t_start,iei (filtered),inst_freq (filtered)
0,0.010,NaN,NaN
1,0.010,NaN,NaN
2,0.010,NaN,NaN
3,0.010,NaN,NaN
4,0.010,NaN,NaN
5,21.551,21.541,0.046423
6,21.551,NaN,NaN


## Step 3: Aggregate stats table

Compute count, min, max, mean, std, sem, cv over filtered iei and inst_freq. Add n_original and context columns.

In [11]:
table = aggregate_intv_stats(iei, inst_freq, n_original)
parsed = parse_rel_path(REL_PATH)
table["original_column"] = TIME_COL
table["roi_id"] = ROI_ID
table["grandparent"] = parsed["grandparent"]
table["parent"] = parsed["parent"]
table["tif_file"] = parsed["tif_file"]
table["event_type"] = EVENT_TYPE
table

,count,min,max,mean,std,sem,cv,n_original,original_column,roi_id,grandparent,parent,tif_file,event_type
iei,1,21.541000,21.541000,21.541000,NaN,NaN,NaN,6,t_start,1,14d Saline,20251020,20251020_A100_0013.tif,baseline_rise
inst_freq,1,0.046423,0.046423,0.046423,NaN,NaN,NaN,6,t_start,1,14d Saline,20251020,20251020_A100_0013.tif,baseline_rise


## Full pipeline: intv_stats()

Run the complete algorithm (same as calling `intv_stats()` from `intv_stats.py`).

---

## Future: integration with plot_pool_widget and plot_pool_app

Once the intv_stats algorithm is finalized, we will plan how to incorporate it into the NiceGUI/Plotly GUI in `plot_pool_widget/` and `plot_pool_app/`. No edits to those modules are made in this notebook.

In [12]:
# Use the actual module when available
try:
    from nicewidgets.plot_pool_widget.algorithms.intv_stats import intv_stats
    result = intv_stats(df_master, TIME_COL, ROI_ID, REL_PATH, EVENT_TYPE)
except ImportError:
    # Fallback: manual pipeline
    df_f = filter_for_intv_stats(df_master, ROI_ID, REL_PATH, EVENT_TYPE)
    ts = df_f[TIME_COL] if TIME_COL in df_f.columns else pd.Series(dtype=float)
    iei, inst_freq = compute_iei_and_inst_freq(ts)
    iei, inst_freq, n_original = filter_zero_iei(iei, inst_freq)
    table = aggregate_intv_stats(iei, inst_freq, n_original)
    parsed = parse_rel_path(REL_PATH)
    table["original_column"] = TIME_COL
    table["roi_id"] = ROI_ID
    table["grandparent"] = parsed["grandparent"]
    table["parent"] = parsed["parent"]
    table["tif_file"] = parsed["tif_file"]
    table["event_type"] = EVENT_TYPE
    result = {"metadata": "", "table": table, "iei": iei, "inst_freq": inst_freq}

print(result["metadata"])
result["table"]

Interval statistics for 't_start'
  - filtering by roi_id = '1'
  - filtering by rel_path = '14d Saline/20251020/20251020_A100_0013.tif'
  - filtering by event_type = 'baseline_rise'


,count,min,max,mean,std,sem,cv,n_original,original_column,roi_id,grandparent,parent,tif_file,event_type
iei,1,21.541000,21.541000,21.541000,NaN,NaN,NaN,6,t_start,1,14d Saline,20251020,20251020_A100_0013.tif,baseline_rise
inst_freq,1,0.046423,0.046423,0.046423,NaN,NaN,NaN,6,t_start,1,14d Saline,20251020,20251020_A100_0013.tif,baseline_rise


## Batch: all (rel_path, event_type) after roi_id filter

Run intv_stats for every unique (rel_path, event_type) in the roi-filtered dataframe. Produces a summary table across all files.

In [11]:
try:
    from nicewidgets.plot_pool_widget.algorithms.intv_stats import intv_stats_batch
    batch = intv_stats_batch(df_master, TIME_COL, ROI_ID)
except ImportError:
    df_roi = df_master[df_master["roi_id"].astype(str) == str(ROI_ID)]
    pairs = df_roi[["rel_path", "event_type"]].drop_duplicates().itertuples(index=False)
    tables = []
    for rp, et in pairs:
        df_f = filter_for_intv_stats(df_master, ROI_ID, rp, str(et))
        ts = df_f[TIME_COL] if TIME_COL in df_f.columns else pd.Series(dtype=float)
        iei, inst_freq = compute_iei_and_inst_freq(ts)
        iei, inst_freq, n_original = filter_zero_iei(iei, inst_freq)
        t = aggregate_intv_stats(iei, inst_freq, n_original)
        p = parse_rel_path(rp)
        t["original_column"] = TIME_COL
        t["roi_id"] = ROI_ID
        t["grandparent"] = p["grandparent"]
        t["parent"] = p["parent"]
        t["tif_file"] = p["tif_file"]
        t["event_type"] = et
        t["stat_type"] = t.index
        tables.append(t.reset_index(drop=True))
    batch = pd.concat(tables, ignore_index=True) if tables else pd.DataFrame()

print(f"Batch summary (roi_id={ROI_ID})")
batch

Batch summary (roi_id=1)


/Users/cudmore/Sites/kymflow_outer/nicewidgets/.venv/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/cudmore/Sites/kymflow_outer/nicewidgets/.venv/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,count,min,max,mean,std,sem,cv,original_column,roi_id,grandparent,parent,tif_file,event_type,stat_type
0,6.0,0.000000,21.541000,3.590167,8.794076,3.590167,2.449490,t_start,1,14d Saline,20251020,20251020_A100_0013.tif,baseline_rise,iei
1,6.0,0.046423,inf,inf,NaN,NaN,NaN,t_start,1,14d Saline,20251020,20251020_A100_0013.tif,baseline_rise,inst_freq
2,12.0,0.458000,11.562000,2.703333,3.141924,0.906995,1.162241,t_start,1,14d Saline,20251020,20251020_A100_0013.tif,nan_gap,iei
3,12.0,0.086490,2.183406,0.812555,0.632301,0.182529,0.778164,t_start,1,14d Saline,20251020,20251020_A100_0013.tif,nan_gap,inst_freq
4,9.0,0.494000,6.328000,2.410556,2.095698,0.698566,0.869384,t_start,1,14d Saline,20251020,20251020_A100_0013.tif,baseline_drop,iei
5,9.0,0.158028,2.024291,0.851180,0.663950,0.221317,0.780035,t_start,1,14d Saline,20251020,20251020_A100_0013.tif,baseline_drop,inst_freq
6,15.0,0.299000,11.440000,2.088467,2.887573,0.745568,1.382628,t_start,1,14d Saline,20251104,20251104_A108_0013.tif,nan_gap,iei
7,15.0,0.087413,3.344482,1.274801,1.001351,0.258548,0.785496,t_start,1,14d Saline,20251104,20251104_A108_0013.tif,nan_gap,inst_freq
8,6.0,0.010000,10.196000,4.110833,3.403636,1.389529,0.827967,t_start,1,14d Saline,20251104,20251104_A108_0013.tif,baseline_drop,iei
9,6.0,0.098078,100.000000,16.877517,40.721646,16.624543,2.412775,t_start,1,14d Saline,20251104,20251104_A108_0013.tif,baseline_drop,inst_freq
